# DS-I Africa Deep Learning Course - 4 July 2023
## Mandla Gwetu (mvgcollab@gmail.com)

In [1]:
import numpy as np
from keras import Sequential, layers, optimizers, models
from sklearn.model_selection import train_test_split
import os

## Many to One Sequence Generator

In [2]:
num_max = 105
s_size = 5
n = num_max - s_size # number of instances
f_size = 1
o_size = 1

In [3]:
def many2one_gen():
  file_x = open('many2one_x.txt','w')
  file_y = open('many2one_y.txt','w')
  for i in range(s_size,num_max):
    for j in range(i-s_size,i):
      file_x.write(str(j)+' ')
    if i!=num_max-1:
      file_x.write('\n')
    if i==num_max-1:
      file_y.write(str(i))
    else:
      file_y.write(str(i)+'\n')
  file_x.close()
  file_y.close()

In [4]:
many2one_gen()

In [5]:
h_state_size = 2
model1 = Sequential()
model1.add(layers.SimpleRNN(h_state_size, input_shape=(s_size, f_size), activation='relu', return_sequences=False))
model1.add(layers.Dense(o_size))
model1.compile(loss="mean_squared_error", optimizer='adam')
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 2)                 8         
                                                                 
 dense (Dense)               (None, 1)                 3         
                                                                 
Total params: 11
Trainable params: 11
Non-trainable params: 0
_________________________________________________________________


In [6]:
X1 = np.loadtxt('many2one_x.txt')
print(X1.shape)
X1 = X1.reshape(n,s_size,f_size)
print(X1.shape)

(100, 5)
(100, 5, 1)


In [7]:
y1 = np.loadtxt('many2one_y.txt')
print(y1.shape)
y1 = y1.reshape(n,o_size)
print(y1.shape)

(100,)
(100, 1)


In [8]:
test_prop = 0.25
X1_train, X1_test, y1_train, y1_test = train_test_split(X1,y1,random_state=1,test_size=test_prop,shuffle=True)
#path1 = 'model1'
#if not os.path.exists(path1):
model1.fit(X1_train, y1_train, epochs=1000, verbose=2)
#else:
  #model1 = models.load_model(path1)

Epoch 1/1000
3/3 - 1s - loss: 1431.6704 - 957ms/epoch - 319ms/step
Epoch 2/1000
3/3 - 0s - loss: 906.3630 - 8ms/epoch - 3ms/step
Epoch 3/1000
3/3 - 0s - loss: 493.6974 - 8ms/epoch - 3ms/step
Epoch 4/1000
3/3 - 0s - loss: 235.1012 - 7ms/epoch - 2ms/step
Epoch 5/1000
3/3 - 0s - loss: 72.5304 - 8ms/epoch - 3ms/step
Epoch 6/1000
3/3 - 0s - loss: 9.1750 - 9ms/epoch - 3ms/step
Epoch 7/1000
3/3 - 0s - loss: 4.7730 - 9ms/epoch - 3ms/step
Epoch 8/1000
3/3 - 0s - loss: 26.0236 - 9ms/epoch - 3ms/step
Epoch 9/1000
3/3 - 0s - loss: 47.9870 - 8ms/epoch - 3ms/step
Epoch 10/1000
3/3 - 0s - loss: 56.1699 - 9ms/epoch - 3ms/step
Epoch 11/1000
3/3 - 0s - loss: 48.4355 - 9ms/epoch - 3ms/step
Epoch 12/1000
3/3 - 0s - loss: 32.5656 - 9ms/epoch - 3ms/step
Epoch 13/1000
3/3 - 0s - loss: 15.5835 - 8ms/epoch - 3ms/step
Epoch 14/1000
3/3 - 0s - loss: 4.8017 - 7ms/epoch - 2ms/step
Epoch 15/1000
3/3 - 0s - loss: 0.7436 - 8ms/epoch - 3ms/step
Epoch 16/1000
3/3 - 0s - loss: 1.6765 - 8ms/epoch - 3ms/step
Epoch 17/1000

In [9]:
wx1 = model1.get_weights()[0]
wh1 = model1.get_weights()[1]
bh1 = model1.get_weights()[2]
wy1 = model1.get_weights()[3]
by1 = model1.get_weights()[4]

print('Input Weights:', "==========================", wx1,sep='\n')
print('Hidden State Weights:', "==========================", wh1,sep='\n')
print('Hidden State Biases:', "==========================", bh1,sep='\n')
print('Output Weights:', "==========================", wy1,sep='\n')
print('Output Biases:', "==========================", by1,sep='\n')

Input Weights:
[[1.4284467  0.20463225]]
Hidden State Weights:
[[ 1.0089834   0.08208359]
 [-0.08295486  0.9821517 ]]
Hidden State Biases:
[ 0.2004796  -0.16741337]
Output Weights:
[[ 0.5192598]
 [-1.23761  ]]
Output Biases:
[0.17434186]


In [10]:
y1_test_ = np.rint(model1.predict(X1_test))
acc1 = np.sum(y1_test == y1_test_)/len(y1_test)*100
print('Accuraccy:',acc1,'%')
print(y1_test[:5])
print(y1_test_[:5])

1/1 [==============================] - 0s 154ms/step
Accuraccy: 100.0 %
[[85.]
 [89.]
 [38.]
 [86.]
 [98.]]
[[85.]
 [89.]
 [38.]
 [86.]
 [98.]]


In [11]:
#model1.save(path1)

## One to Many Sequence Generator

In [12]:
s_size = 1
o_size = 5
def one2many_gen():
  file_x = open('one2many_x.txt','w')
  file_y = open('one2many_y.txt','w')
  for i in range(num_max-o_size):
    for j in range(i+1,i+1+o_size):
      file_y.write(str(j)+' ')
      if j==i+o_size and i!=num_max-o_size-1:
        file_y.write('\n')
    if i==num_max-o_size-1:
      file_x.write(str(i))
    else:
      file_x.write(str(i)+'\n')
  file_x.close()
  file_y.close()

In [13]:
one2many_gen()

In [14]:
h_state_size = 2
model2 = Sequential()
model2.add(layers.SimpleRNN(h_state_size, input_shape=(s_size, f_size), activation='relu', return_sequences=False))
model2.add(layers.Dense(o_size))
model2.compile(loss="mean_squared_error", optimizer='adam')
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 2)                 8         
                                                                 
 dense_1 (Dense)             (None, 5)                 15        
                                                                 
Total params: 23
Trainable params: 23
Non-trainable params: 0
_________________________________________________________________


In [15]:
X2 = np.loadtxt('one2many_x.txt')
print(X2.shape)
X2 = X2.reshape(n,s_size,f_size)
print(X2.shape)

(100,)
(100, 1, 1)


In [16]:
y2 = np.loadtxt('one2many_y.txt')
print(y2.shape)
y2 = y2.reshape(n,o_size)
print(y2.shape)

(100, 5)
(100, 5)


In [23]:
test_prop = 0.25
X2_train, X2_test, y2_train, y2_test = train_test_split(X2,y2,random_state=1,test_size=test_prop,shuffle=True)
#path2 = 'model2'
#if not os.path.exists(path2):
model2.fit(X2_train, y2_train, epochs=1000, verbose=2)
#else:
  #model2 = models.load_model(path2)


Epoch 1/1000
3/3 - 0s - loss: 0.5962 - 9ms/epoch - 3ms/step
Epoch 2/1000
3/3 - 0s - loss: 0.5953 - 9ms/epoch - 3ms/step
Epoch 3/1000
3/3 - 0s - loss: 0.5947 - 7ms/epoch - 2ms/step
Epoch 4/1000
3/3 - 0s - loss: 0.5938 - 7ms/epoch - 2ms/step
Epoch 5/1000
3/3 - 0s - loss: 0.5931 - 7ms/epoch - 2ms/step
Epoch 6/1000
3/3 - 0s - loss: 0.5924 - 8ms/epoch - 3ms/step
Epoch 7/1000
3/3 - 0s - loss: 0.5917 - 8ms/epoch - 3ms/step
Epoch 8/1000
3/3 - 0s - loss: 0.5909 - 8ms/epoch - 3ms/step
Epoch 9/1000
3/3 - 0s - loss: 0.5903 - 7ms/epoch - 2ms/step
Epoch 10/1000
3/3 - 0s - loss: 0.5893 - 7ms/epoch - 2ms/step
Epoch 11/1000
3/3 - 0s - loss: 0.5885 - 8ms/epoch - 3ms/step
Epoch 12/1000
3/3 - 0s - loss: 0.5876 - 7ms/epoch - 2ms/step
Epoch 13/1000
3/3 - 0s - loss: 0.5869 - 7ms/epoch - 2ms/step
Epoch 14/1000
3/3 - 0s - loss: 0.5865 - 6ms/epoch - 2ms/step
Epoch 15/1000
3/3 - 0s - loss: 0.5855 - 7ms/epoch - 2ms/step
Epoch 16/1000
3/3 - 0s - loss: 0.5849 - 7ms/epoch - 2ms/step
Epoch 17/1000
3/3 - 0s - loss: 0.

In [18]:
wx2 = model2.get_weights()[0]
wh2 = model2.get_weights()[1]
bh2 = model2.get_weights()[2]
wy2 = model2.get_weights()[3]
by2 = model2.get_weights()[4]

In [19]:
print('Input Weights:', "==========================", wx2,sep='\n')
print('Hidden State Weights:', "==========================", wh2,sep='\n')
print('Hidden State Biases:', "==========================", bh2,sep='\n')
print('Output Weights:', "==========================", wy2,sep='\n')
print('Output Biases:', "==========================", by2,sep='\n')

Input Weights:
[[ 1.7474065  -0.48740304]]
Hidden State Weights:
[[-0.79886556 -0.6015096 ]
 [ 0.6015096  -0.7988656 ]]
Hidden State Biases:
[0.80149096 0.        ]
Output Weights:
[[ 0.5700033   0.5797638   0.58942366  0.5929188   0.60527086]
 [-0.4443769  -0.23056084 -0.12939239  0.43714702  0.3188367 ]]
Output Biases:
[0.801637   0.6807312  0.56858087 1.1674669  0.75329155]


In [20]:
y2_test_ = np.rint(model2.predict(X2_test))
acc2 = np.sum(y2_test == y2_test_)/(o_size*len(y2_test))*100
print('Accuraccy:',acc2,'%')
print(y2_test[:5])
print(y2_test_[:5])

1/1 [==============================] - 0s 155ms/step
Accuraccy: 54.400000000000006 %
[[81. 82. 83. 84. 85.]
 [85. 86. 87. 88. 89.]
 [34. 35. 36. 37. 38.]
 [82. 83. 84. 85. 86.]
 [94. 95. 96. 97. 98.]]
[[ 81.  82.  83.  85.  86.]
 [ 85.  86.  88.  89.  90.]
 [ 34.  35.  35.  36.  36.]
 [ 82.  83.  84.  86.  87.]
 [ 94.  95.  97.  98. 100.]]


In [21]:
#model2.save(path2)

## Exercise
Separate this notebook into two separate notebooks: focusing on many-to-one and one-to-many examples, respectively. You will notice that the model does not always converge - this is likely due to the lack of feature normalisation in the dataset. Try and see if you can find models with training losses that are less than 0.001 at the end of all the epochs. Save these models and draw a diagram that represents the components and weights. Can you trace one input through the model and verify the output?